<a href="https://colab.research.google.com/github/swiatej/ds4/blob/main/sql_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# The code below sets up our environment. We are running Hive on Spark.

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!tar xf spark-3.3.1-bin-hadoop2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

We will carry out simple data analysis in Hive first, before moving on to PySpark

## Canada

In [ ]:
# Creating table
spark.sql('create table if not exists youtube_ca \
          (id int,video_id string,title string,publishedAt string,channelId string, channelTitle string,categoryId string,trending_date string,tags string, view_count int,likes int,dislikes int,comment_count int,thumbnail_link string,comments_disabled string,ratings_disabled string,description string) \
         row format delimited fields terminated by ","\
         stored as textfile') 
# Loading data in from .csv
spark.sql('load data inpath "/content/drive/MyDrive/youtube analysis/CA_data_youtube_1.csv" into table youtube_ca')
# Creating new table 
spark.sql('CREATE TABLE youtube_canada LIKE youtube_ca')
# Loading in data from old table, omitting rows which are all NULL
spark.sql('INSERT OVERWRITE TABLE youtube_canada SELECT * FROM youtube_ca WHERE id is not null and  video_id is not null and title is not null and publishedAt is not null and channelId is not null and  channelTitle is not null and categoryId is not null and trending_date is not null and tags is not null and  view_count is not null and likes is not null and dislikes is not null and comment_count is not null and thumbnail_link is not null and comments_disabled is not null and ratings_disabled is not null and description is not null')


DataFrame[]

In [ ]:
# Looking up the top most-liked videos
spark.sql('select * from youtube_canada order by likes desc limit 10').show(truncate=False)

+-----+-----------+--------------------------------------------------+--------------------+------------------------+--------------+-----------+--------------------+-----------------------------------------+----------+--------+--------+-------------+----------------------------------------------+-----------------+----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# How many days did the top-liked videos stay on top?
spark.sql('SELECT title, channelTitle,COUNT(trending_date) as no_of_days FROM youtube_canada GROUP BY title,channelTitle ORDER BY no_of_days DESC').show(truncate = False)

+------------------------------------------------------------------------------------------+-------------+----------+
|title                                                                                     |channelTitle |no_of_days|
+------------------------------------------------------------------------------------------+-------------+----------+
|Starlink Mission                                                                          |SpaceX       |105       |
|I Designed Custom Minecraft Bosses...                                                     |Daniel Krafft|25        |
|Extreme Try Not To Laugh Challenge!                                                       |Beast Reacts |19        |
|Most Oddly Satisfying Video to watch before sleep                                         |SSSniperWolf |19        |
|The FUNNIEST FAKE Minecraft Speedruns EVER                                                |Ayundaru     |18        |
|Trying Tik Tok Life Hacks to see if they work          

In [ ]:
# Most disliked videos - note some of them are the same as the 'most-liked' ones
spark.sql('select * from youtube_canada order by dislikes desc limit 10').show(truncate=False)

+----+-----------+-----------------------------------------------+--------------------+------------------------+--------------+-----------+--------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------+--------+-------------+----------------------------------------------+-----------------+----------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Top most-viewed videos
spark.sql('select * from youtube_canada order by view_count desc limit 10').show(truncate=False)

+------+-----------+---------------------------------------+--------------------+------------------------+--------------+-----------+--------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------+--------+-------------+----------------------------------------------+-----------------+----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Top categories along with the number of videos in the category and average viewcount per video, sorted by average view count
spark.sql('SELECT categoryid, AVG(view_count) as avg_views, COUNT(video_id) as no_of_videos \
FROM youtube_canada \
GROUP BY categoryid \
ORDER BY avg_views DESC \
LIMIT 10 ').show(truncate = False)

+-----------+------------------+------------+
|categandyid|avg_views         |no_of_videos|
+-----------+------------------+------------+
|10         |4724986.975840238 |21482       |
|24         |2730718.7534258505|34444       |
|28         |2333061.902219963 |5901        |
|1          |2231661.498248073 |5708        |
|29         |2226000.15625     |96          |
|22         |2193916.8125658864|14229       |
|23         |2056217.5084452585|9828        |
|20         |1730197.0402017778|32511       |
|17         |1652891.027498184 |19274       |
|27         |1596150.3618121638|4834        |
+-----------+------------------+------------+



In [ ]:
# Top categories along with the number of videos, average likes / dislikes per video, sorted by number of videos
spark.sql('SELECT categoryid, AVG(likes) as avglikes, AVG(dislikes) as avgdislikes,COUNT(video_id) as no_of_videos FROM youtube_canada \
GROUP BY categoryid \
ORDER BY no_of_videos DESC \
LIMIT 10').show(truncate=False) 


+-----------+------------------+------------------+------------+
|categandyid|avglikes          |avgdislikes       |no_of_videos|
+-----------+------------------+------------------+------------+
|24         |147122.90683428172|1756.7754616188597|34444       |
|20         |95503.5845098582  |1090.4344375749747|32511       |
|10         |330005.61623684946|4195.60832324737  |21482       |
|17         |42759.156532115805|760.0422330600809 |19274       |
|22         |120901.93007238737|2497.931969920585 |14229       |
|23         |135226.97863247863|1982.8930606430606|9828        |
|28         |100119.70954075581|1537.751228605321 |5901        |
|1          |99773.71478626489 |1025.6119481429573|5708        |
|25         |19447.736551478447|1071.3599928749554|5614        |
|26         |72016.65421597633 |1613.3004807692307|5408        |
+-----------+------------------+------------------+------------+



In [ ]:
# Top channels sorted by distinct number of videos, along with their average likes,dislikes
spark.sql('SELECT channeltitle, AVG(likes) as avglikes, AVG(dislikes) as avgdislikes, COUNT(DISTINCT video_id) as no_of_videos \
FROM youtube_canada \
GROUP BY channeltitle \
ORDER BY no_of_videos DESC \
LIMIT 10').show()


+--------------------+------------------+------------------+------------+
|        channeltitle|          avglikes|       avgdislikes|no_of_videos|
+--------------------+------------------+------------------+------------+
|                 NBA|22326.526795895097| 457.2337514253136|         241|
|         DAZN Canada|1634.3926499032882|23.328820116054157|         126|
|                 NHL| 2179.285714285714| 86.54761904761905|         101|
|             SSundee| 259522.7842639594|3017.6548223350255|          83|
|           FORMULA 1|101466.04683840749|1137.3302107728337|          81|
|UFC - Ultimate Fi...| 37577.67576791809| 741.5938566552901|          77|
|           SPORTSNET| 4827.667560321715|  87.5201072386059|          76|
|                 NFL|110537.77678571429|2035.4791666666667|          73|
|              SMTOWN| 993685.7375690608|10352.185082872928|          69|
|      Genshin Impact| 189091.5876923077| 502.6307692307692|          67|
+--------------------+----------------

In [ ]:
spark.sql('SELECT channeltitle, AVG(likes) as avglikes, AVG(dislikes) as avgdislikes, AVG(view_count) as avgviews,COUNT(DISTINCT video_id) as no_of_videos FROM youtube_canada \
GROUP BY channeltitle \
HAVING avgviews > 1000000 AND no_of_videos > 20 \
ORDER BY avglikes DESC \
LIMIT 10').show()


+-----------------+------------------+------------------+--------------------+------------+
|     channeltitle|          avglikes|       avgdislikes|            avgviews|no_of_videos|
+-----------------+------------------+------------------+--------------------+------------+
|        BLACKPINK|2906195.1739130435| 43091.19927536232|3.1349952510869566E7|          53|
|      HYBE LABELS|2426532.8907563025|  8464.83193277311|2.1509151403361343E7|          49|
|          MrBeast|1938847.1173594133|14166.036674816625| 2.873162808801956E7|          55|
|        BANGTANTV|        1914764.25| 7322.662698412699|1.1371754992063493E7|          53|
|           SMTOWN| 993685.7375690608|10352.185082872928|1.6741165080110498E7|          69|
|JYP Entertainment| 974165.5575757576|10509.633333333333|1.6176691245454546E7|          63|
| BillieEilishVEVO| 827902.9612903226|16276.090322580645|   9437203.232258065|          27|
|       Mark Rober| 734163.0891719746| 5000.095541401274|1.2501924401273886E7|  

In [ ]:
spark.sql('SELECT channeltitle, AVG(likes) as avglikes, AVG(dislikes) as avgdislikes, AVG(view_count) as avgviews,COUNT(DISTINCT video_id) as no_of_videos \
FROM youtube_canada \
GROUP BY channeltitle \
HAVING avgviews > 1000000 \
AND no_of_videos > 20 \
ORDER BY avgdislikes DESC \
LIMIT 10').show()

+-----------------+------------------+------------------+--------------------+------------+
|     channeltitle|          avglikes|       avgdislikes|            avgviews|no_of_videos|
+-----------------+------------------+------------------+--------------------+------------+
|        BLACKPINK|2906195.1739130435| 43091.19927536232|3.1349952510869566E7|          53|
|        Jake Paul| 60098.08280254777|21144.808917197453|   1836962.872611465|          28|
| BillieEilishVEVO| 827902.9612903226|16276.090322580645|   9437203.232258065|          27|
|          MrBeast|1938847.1173594133|14166.036674816625| 2.873162808801956E7|          55|
|   Dixie D'Amelio|334535.25165562914|11043.788079470198|   5036954.337748344|          25|
|JYP Entertainment| 974165.5575757576|10509.633333333333|1.6176691245454546E7|          63|
|           SMTOWN| 993685.7375690608|10352.185082872928|1.6741165080110498E7|          69|
|      jeffreestar| 67374.21393034825| 9415.069651741294|   1491818.985074627|  

In [ ]:
# most comments - looking at channels
spark.sql('SELECT channeltitle, AVG(likes) as avglikes, AVG(dislikes) as avgdislikes, AVG(view_count) as avgviews,AVG(comment_count) as avgcomment,COUNT(DISTINCT video_id) as no_of_videos \
FROM youtube_canada \
GROUP BY channeltitle \
HAVING avgviews > 1000000 \
AND no_of_videos > 20 \
ORDER BY avgcomment DESC \
LIMIT 10').show()

+-----------------+------------------+------------------+--------------------+------------------+------------+
|     channeltitle|          avglikes|       avgdislikes|            avgviews|        avgcomment|no_of_videos|
+-----------------+------------------+------------------+--------------------+------------------+------------+
|      HYBE LABELS|2426532.8907563025|  8464.83193277311|2.1509151403361343E7| 458762.7226890756|          49|
|        BLACKPINK|2906195.1739130435| 43091.19927536232|3.1349952510869566E7|445936.59420289856|          53|
|           SMTOWN| 993685.7375690608|10352.185082872928|1.6741165080110498E7| 209760.8093922652|          69|
|JYP Entertainment| 974165.5575757576|10509.633333333333|1.6176691245454546E7|180971.48787878788|          63|
|          MrBeast|1938847.1173594133|14166.036674816625| 2.873162808801956E7|116447.45232273839|          55|
|        BANGTANTV|        1914764.25| 7322.662698412699|1.1371754992063493E7| 90027.23015873016|          53|
|

In [ ]:
# least comments but still big channels
spark.sql('SELECT channeltitle, AVG(likes) as avglikes, AVG(dislikes) as avgdislikes, AVG(view_count) as avgviews,AVG(comment_count) as avgcomment,COUNT(DISTINCT video_id) as no_of_videos \
FROM youtube_canada \
GROUP BY channeltitle \
HAVING avgviews > 1000000 \
AND no_of_videos > 20 \
ORDER BY avgcomment asc \
LIMIT 10').show()

+-------------------+------------------+------------------+------------------+------------------+------------+
|       channeltitle|          avglikes|       avgdislikes|          avgviews|        avgcomment|no_of_videos|
+-------------------+------------------+------------------+------------------+------------------+------------+
|              Apple|236180.81463414634| 8982.053658536586|  8523140.03902439|               0.0|          37|
|         Bundesliga| 22393.17924528302| 264.4433962264151|1120568.4056603773| 978.3584905660377|          23|
|               NASA|  83397.7741935484|  1069.47311827957|3161380.4623655914|1019.5913978494624|          37|
|    SQUEEZIE GAMING| 89406.93063583814| 617.8901734104046|1173287.5028901733|1341.4335260115606|          35|
|   Max the Meat Guy|112887.00869565217|1176.3739130434783|1695563.4956521739| 1342.304347826087|          23|
|  Dude Perfect Plus|44583.444444444445|470.59722222222223|1156522.7847222222| 1403.513888888889|          25|
|

In [ ]:
# most comments for a video (do they correlate with most viewed/liked videos) - they do
spark.sql('SELECT channeltitle, title, publishedat,comment_count, likes, dislikes, trending_date \
FROM youtube_canada \
ORDER BY comment_count desc \
LIMIT 10').show(truncate=False)

+--------------+---------------------------------------+--------------------+-------------+--------+--------+--------------------+
|channeltitle  |title                                  |publishedat         |comment_count|likes   |dislikes|trending_date       |
+--------------+---------------------------------------+--------------------+-------------+--------+--------+--------------------+
|HYBE LABELS   |BTS (방탄소년단) 'Butter' Official MV  |2021-05-21T03:46:13Z|6738536      |16021548|150990  |2021-05-30T00:00:00Z|
|Big Hit Labels|BTS (방탄소년단) 'Dynamite' Official MV|2020-08-21T03:58:10Z|6064909      |15735572|714195  |2020-08-28T00:00:00Z|
|HYBE LABELS   |BTS (방탄소년단) 'Butter' Official MV  |2021-05-21T03:46:13Z|5987760      |14202560|51664   |2021-05-23T00:00:00Z|
|Big Hit Labels|BTS (방탄소년단) 'Dynamite' Official MV|2020-08-21T03:58:10Z|5810703      |15460854|674295  |2020-08-27T00:00:00Z|
|Big Hit Labels|BTS (방탄소년단) 'Dynamite' Official MV|2020-08-21T03:58:10Z|5748687      |14678128|576858  

## USA

In [ ]:
spark.sql('create table if not exists youtube_us \
          (id int,video_id string,title string,publishedAt string,channelId string, channelTitle string,categoryId string,trending_date string,tags string, view_count int,likes int,dislikes int,comment_count int,thumbnail_link string,comments_disabled string,ratings_disabled string,description string) \
         row format delimited fields terminated by ","\
         stored as textfile') 

spark.sql('load data inpath "/content/drive/MyDrive/youtube analysis/US_data_youtube_1.csv" into table youtube_us')
spark.sql('CREATE TABLE youtube_usa LIKE youtube_us')

spark.sql('INSERT OVERWRITE TABLE youtube_usa SELECT * FROM youtube_us WHERE id is not null and  video_id is not null and title is not null and publishedAt is not null and channelId is not null and  channelTitle is not null and categoryId is not null and trending_date is not null and tags is not null and  view_count is not null and likes is not null and dislikes is not null and comment_count is not null and thumbnail_link is not null and comments_disabled is not null and ratings_disabled is not null and description is not null')


DataFrame[]

In [ ]:
spark.sql('select * from youtube_usa order by likes desc limit 10').show(truncate=False)

+------+-----------+---------------------------------------+--------------------+------------------------+--------------+----------+--------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# How many days did the top-liked videos stay on top?
spark.sql('SELECT title, channelTitle,COUNT(trending_date) as no_of_days FROM youtube_usa GROUP BY title,channelTitle ORDER BY no_of_days DESC').show(truncate = False)

+---------------------------------------------------------------------------------------------+--------------------+----------+
|title                                                                                        |channelTitle        |no_of_days|
+---------------------------------------------------------------------------------------------+--------------------+----------+
|Starlink Mission                                                                             |SpaceX              |178       |
|Most Oddly Satisfying Video to watch before sleep                                            |SSSniperWolf        |50        |
|Floyd Mayweather vs Logan Paul: Fight goes the distance [Highlights  recap] | CBS Sports HQ  |CBS Sports HQ       |35        |
|Creative People On Another Level                                                             |SSSniperWolf        |35        |
|India claim stunning series win  end Australia's Gabba streak | Vodafone Test Series 2020-21 |cricket.c

In [ ]:
spark.sql('select * from youtube_usa order by dislikes desc limit 10').show(truncate=False)

+----+-----------+-----------------------------------------------+--------------------+------------------------+--------------+----------+--------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------+--------+-------------+----------------------------------------------+-----------------+----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
spark.sql('select * from youtube_usa order by view_count desc limit 10').show(truncate=False)

+------+-----------+-------------------------------------+--------------------+------------------------+------------+----------+--------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------+--------+-------------+----------------------------------------------+-----------------+----------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
spark.sql('SELECT categoryid, AVG(view_count) as avg_views, COUNT(video_id) as no_of_videos \
FROM youtube_usa \
GROUP BY categoryid \
ORDER BY avg_views DESC \
LIMIT 10 ').show(truncate = False)

+----------+------------------+------------+
|categoryid|avg_views         |no_of_videos|
+----------+------------------+------------+
|10        |4477166.513309751 |27987       |
|24        |3049913.5198641038|32672       |
|28        |2591995.4152941178|5100        |
|29        |2449722.447368421 |114         |
|25        |2319410.0800808584|6431        |
|1         |2213262.7040955117|6617        |
|17        |1969061.4005192223|18104       |
|20        |1792941.667775519 |32466       |
|22        |1774310.800575954 |13890       |
|23        |1627778.061661268 |8644        |
+----------+------------------+------------+



In [ ]:
spark.sql('SELECT categoryid, AVG(likes) as avglikes, AVG(dislikes) as avgdislikes,COUNT(video_id) as no_of_videos FROM youtube_usa \
GROUP BY categoryid \
ORDER BY no_of_videos DESC \
LIMIT 10').show(truncate=False) 


+----------+------------------+------------------+------------+
|categoryid|avglikes          |avgdislikes       |no_of_videos|
+----------+------------------+------------------+------------+
|24        |152112.33349657198|1758.3897833006856|32672       |
|20        |98469.52331670055 |1141.141963900696 |32466       |
|10        |296889.4524958016 |3770.1660056454784|27987       |
|17        |49586.64206805126 |870.8666593018118 |18104       |
|22        |104244.62375809935|1959.3008639308855|13890       |
|23        |113794.31027302175|1039.3157103192966|8644        |
|1         |94786.54027504912 |988.185280338522  |6617        |
|25        |29356.179287824598|1500.9244285492148|6431        |
|28        |108658.24470588236|1734.6970588235295|5100        |
|26        |67352.226342711   |1529.886189258312 |4692        |
+----------+------------------+------------------+------------+



In [ ]:
spark.sql('SELECT channeltitle, AVG(likes) as avglikes, AVG(dislikes) as avgdislikes, COUNT(DISTINCT video_id) as no_of_videos \
FROM youtube_usa \
GROUP BY channeltitle \
ORDER BY no_of_videos DESC \
LIMIT 10').show()


+--------------------+------------------+------------------+------------+
|        channeltitle|          avglikes|       avgdislikes|no_of_videos|
+--------------------+------------------+------------------+------------+
|                 NFL| 95515.34733893558| 925.6003734827265|         280|
|                 NBA|           25159.4| 542.5151515151515|         274|
|        SSSniperWolf|160581.51365187715|2181.1245733788396|         115|
|          NBC Sports| 18195.95901639344|362.69877049180326|         114|
|             SSundee| 246099.8267223382| 2719.874739039666|         101|
|                ESPN|18080.714606741574|1240.9865168539325|          89|
| Saturday Night Live| 62550.30396475771|3997.0308370044054|          78|
|              SpaceX| 43112.74792243767| 617.1080332409972|          74|
|      MrBeast Gaming|511542.39756592293| 6645.843813387424|          72|
|UFC - Ultimate Fi...| 38757.48028673835| 742.6810035842294|          70|
+--------------------+----------------

In [ ]:
spark.sql('SELECT channeltitle, AVG(likes) as avglikes, AVG(dislikes) as avgdislikes, AVG(view_count) as avgviews,COUNT(DISTINCT video_id) as no_of_videos FROM youtube_usa \
GROUP BY channeltitle \
HAVING avgviews > 1000000 AND no_of_videos > 20 \
ORDER BY avglikes DESC \
LIMIT 10').show()


+-----------------+------------------+------------------+--------------------+------------+
|     channeltitle|          avglikes|       avgdislikes|            avgviews|no_of_videos|
+-----------------+------------------+------------------+--------------------+------------+
|        BLACKPINK| 3640215.385665529| 39506.82593856655| 4.821493279863481E7|          49|
|      HYBE LABELS|2679478.6592920353| 9485.942477876106|2.3698966044247787E7|          43|
|          MrBeast|1939631.9060402685|14058.422818791947|2.8961469718120806E7|          55|
|        BANGTANTV|1888622.3412162163| 7607.844594594595|      1.1365684125E7|          57|
|           SMTOWN| 1118414.425867508|11847.340694006309| 1.891546006940063E7|          58|
|JYP Entertainment|1036949.9610389611|11065.951298701299|1.7591681246753246E7|          56|
| BillieEilishVEVO| 813609.3817204301|  16424.2311827957|    9534398.17204301|          30|
|       Mark Rober|  737058.296969697| 5056.036363636364|1.2393038951515151E7|  

In [ ]:
spark.sql('SELECT channeltitle, AVG(likes) as avglikes, AVG(dislikes) as avgdislikes, AVG(view_count) as avgviews,COUNT(DISTINCT video_id) as no_of_videos \
FROM youtube_usa \
GROUP BY channeltitle \
HAVING avgviews > 1000000 \
AND no_of_videos > 20 \
ORDER BY avgdislikes DESC \
LIMIT 10').show()

+-----------------+------------------+------------------+--------------------+------------+
|     channeltitle|          avglikes|       avgdislikes|            avgviews|no_of_videos|
+-----------------+------------------+------------------+--------------------+------------+
|        BLACKPINK| 3640215.385665529| 39506.82593856655| 4.821493279863481E7|          49|
|        Jake Paul|61866.616766467065| 22260.40119760479|  1928715.5329341318|          27|
| BillieEilishVEVO| 813609.3817204301|  16424.2311827957|    9534398.17204301|          30|
|          MrBeast|1939631.9060402685|14058.422818791947|2.8961469718120806E7|          55|
|           SMTOWN| 1118414.425867508|11847.340694006309| 1.891546006940063E7|          58|
|JYP Entertainment|1036949.9610389611|11065.951298701299|1.7591681246753246E7|          56|
|   Dixie D'Amelio| 322259.3865030675|10748.858895705522|   4952167.993865031|          25|
|    James Charles| 531334.7374301676|10720.608938547486|   8222064.301675978|  

In [ ]:
# most comments - looking at channels
spark.sql('SELECT channeltitle, AVG(likes) as avglikes, AVG(dislikes) as avgdislikes, AVG(view_count) as avgviews,AVG(comment_count) as avgcomment,COUNT(DISTINCT video_id) as no_of_videos \
FROM youtube_usa \
GROUP BY channeltitle \
HAVING avgviews > 1000000 \
AND no_of_videos > 20 \
ORDER BY avgcomment DESC \
LIMIT 10').show()

+-----------------+------------------+------------------+--------------------+------------------+------------+
|     channeltitle|          avglikes|       avgdislikes|            avgviews|        avgcomment|no_of_videos|
+-----------------+------------------+------------------+--------------------+------------------+------------+
|        BLACKPINK| 3640215.385665529| 39506.82593856655| 4.821493279863481E7| 645052.6621160409|          49|
|      HYBE LABELS|2679478.6592920353| 9485.942477876106|2.3698966044247787E7|506934.78761061945|          43|
|           SMTOWN| 1118414.425867508|11847.340694006309| 1.891546006940063E7|246901.20504731862|          58|
|JYP Entertainment|1036949.9610389611|11065.951298701299|1.7591681246753246E7|200546.69155844155|          56|
|          MrBeast|1939631.9060402685|14058.422818791947|2.8961469718120806E7|118513.57718120805|          55|
|        BANGTANTV|1888622.3412162163| 7607.844594594595|      1.1365684125E7| 87872.63175675676|          57|
|

In [ ]:
# least comments but still big channels
spark.sql('SELECT channeltitle, AVG(likes) as avglikes, AVG(dislikes) as avgdislikes, AVG(view_count) as avgviews,AVG(comment_count) as avgcomment,COUNT(DISTINCT video_id) as no_of_videos \
FROM youtube_usa \
GROUP BY channeltitle \
HAVING avgviews > 1000000 \
AND no_of_videos > 20 \
ORDER BY avgcomment asc \
LIMIT 10').show()

+--------------------+------------------+------------------+------------------+------------------+------------+
|        channeltitle|          avglikes|       avgdislikes|          avgviews|        avgcomment|no_of_videos|
+--------------------+------------------+------------------+------------------+------------------+------------+
|               Apple| 232377.4928229665| 9157.272727272728| 9391955.746411484|               0.0|          38|
|Ligue 1 Uber Eats...| 23844.72131147541|402.40983606557376| 1487942.762295082| 889.1311475409836|          25|
|           The Voice|21414.361904761903|191.83809523809524|1122325.3523809523|1086.3904761904762|          25|
|                NASA| 85325.99411764706|1122.8882352941177| 3227609.817647059|            1140.8|          32|
|                Hulu| 8544.782178217822|105.00990099009901| 4618822.683168317|1221.8712871287128|          21|
|   Dude Perfect Plus| 46292.27922077922| 495.0584415584416|1221459.4935064935|1412.2987012987012|      

In [ ]:
# most comments for a video (do they correlate with most viewed/liked videos) - they do
spark.sql('SELECT channeltitle, title, publishedat,comment_count, likes, dislikes, trending_date \
FROM youtube_usa \
ORDER BY comment_count desc \
LIMIT 10').show(truncate=False)

+--------------+---------------------------------------+--------------------+-------------+--------+--------+--------------------+
|channeltitle  |title                                  |publishedat         |comment_count|likes   |dislikes|trending_date       |
+--------------+---------------------------------------+--------------------+-------------+--------+--------+--------------------+
|HYBE LABELS   |BTS (방탄소년단) 'Butter' Official MV  |2021-05-21T03:46:13Z|6738537      |16021534|150989  |2021-05-30T00:00:00Z|
|Big Hit Labels|BTS (방탄소년단) 'Dynamite' Official MV|2020-08-21T03:58:10Z|6065230      |15735533|714194  |2020-08-28T00:00:00Z|
|HYBE LABELS   |BTS (방탄소년단) 'Butter' Official MV  |2021-05-21T03:46:13Z|5987770      |14202518|51663   |2021-05-23T00:00:00Z|
|Big Hit Labels|BTS (방탄소년단) 'Dynamite' Official MV|2020-08-21T03:58:10Z|5810680      |15460822|674291  |2020-08-27T00:00:00Z|
|Big Hit Labels|BTS (방탄소년단) 'Dynamite' Official MV|2020-08-21T03:58:10Z|5748638      |14678048|576852  

## Great Britain

In [ ]:
spark.sql('create table if not exists youtube_greatb \
          (id int,video_id string,title string,publishedAt string,channelId string, channelTitle string,categoryId string,trending_date string,tags string, view_count int,likes int,dislikes int,comment_count int,thumbnail_link string,comments_disabled string,ratings_disabled string,description string) \
         row format delimited fields terminated by ","\
         stored as textfile') 

spark.sql('load data inpath "/content/drive/MyDrive/youtube analysis/GB_data_youtube_1.csv" into table youtube_greatb')
spark.sql('CREATE TABLE youtube_gb LIKE youtube_greatb')

spark.sql('INSERT OVERWRITE TABLE youtube_gb SELECT * FROM youtube_greatb WHERE id is not null and  video_id is not null and title is not null and publishedAt is not null and channelId is not null and  channelTitle is not null and categoryId is not null and trending_date is not null and tags is not null and  view_count is not null and likes is not null and dislikes is not null and comment_count is not null and thumbnail_link is not null and comments_disabled is not null and ratings_disabled is not null and description is not null')


DataFrame[]

In [ ]:
spark.sql('select * from youtube_gb order by likes desc limit 10').show(truncate=False)

+----+-----------+-----------------------------------------------+--------------------+------------------------+--------------+----------+--------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------+--------+-------------+----------------------------------------------+-----------------+----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# How many days did the top-liked videos stay on top?
spark.sql('SELECT title, channelTitle,COUNT(trending_date) as no_of_days FROM youtube_gb GROUP BY title,channelTitle ORDER BY no_of_days DESC').show(truncate = False)

+-------------------------------------------------------------------------------------------------+--------------------+----------+
|title                                                                                            |channelTitle        |no_of_days|
+-------------------------------------------------------------------------------------------------+--------------------+----------+
|Starlink Mission                                                                                 |SpaceX              |15        |
|HIGHLIGHTS | Los Angeles Lakers vs Miami Heat                                                    |Los Angeles Lakers  |15        |
|Da Beatfreakz x DigDat x Dutchavelli x B Young - 808 [Music Video] | GRM Daily                   |GRM Daily           |8         |
|Diversity take to the stage with POWERFUL Black Lives Matter performance | Semi-Finals | BGT 2020|Britain's Got Talent|8         |
|Little Mix - Holiday (Official Video)                                      

In [ ]:
spark.sql('select * from youtube_gb order by dislikes desc limit 10').show(truncate=False)

+----+-----------+-----------------------------------------------+--------------------+------------------------+--------------+----------+--------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------+--------+-------------+----------------------------------------------+-----------------+----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
spark.sql('select * from youtube_gb order by view_count desc limit 10').show(truncate=False)

+----+-----------+-----------------------------------------------+--------------------+------------------------+--------------+----------+--------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------+--------+-------------+----------------------------------------------+-----------------+----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
spark.sql('SELECT categoryid, AVG(view_count) as avg_views, COUNT(video_id) as no_of_videos \
FROM youtube_gb \
GROUP BY categoryid \
ORDER BY avg_views DESC \
LIMIT 10 ').show(truncate = False)

+----------+------------------+------------+
|categoryid|avg_views         |no_of_videos|
+----------+------------------+------------+
|10        |4386661.821923229 |2527        |
|20        |2524558.2397447582|1097        |
|28        |2262507.8688946017|778         |
|24        |1794359.676913618 |3253        |
|1         |1792189.457364341 |387         |
|22        |1711465.8142734307|1163        |
|23        |1204920.6054931337|801         |
|27        |1002478.6742209631|353         |
|25        |995569.1018276763 |383         |
|26        |989568.0581395349 |516         |
+----------+------------------+------------+



In [ ]:
spark.sql('SELECT categoryid, AVG(likes) as avglikes, AVG(dislikes) as avgdislikes,COUNT(video_id) as no_of_videos FROM youtube_gb \
GROUP BY categoryid \
ORDER BY no_of_videos DESC \
LIMIT 10').show(truncate=False) 


+----------+------------------+------------------+------------+
|categoryid|avglikes          |avgdislikes       |no_of_videos|
+----------+------------------+------------------+------------+
|24        |117610.78051029818|2322.516753765755 |3253        |
|17        |23126.392405063292|679.9020652898068 |3002        |
|10        |353055.5916106055 |9455.037593984962 |2527        |
|22        |109477.83748925194|3467.484092863285 |1163        |
|20        |127300.77301731997|3587.950774840474 |1097        |
|23        |108150.94756554307|1237.3320848938827|801         |
|28        |82692.07583547558 |2943.1799485861184|778         |
|26        |62437.15503875969 |1950.4864341085272|516         |
|1         |94166.87596899224 |1920.0413436692506|387         |
|25        |15402.514360313317|2543.77545691906  |383         |
+----------+------------------+------------------+------------+



In [ ]:
spark.sql('SELECT channeltitle, AVG(likes) as avglikes, AVG(dislikes) as avgdislikes, COUNT(DISTINCT video_id) as no_of_videos \
FROM youtube_gb \
GROUP BY channeltitle \
ORDER BY no_of_videos DESC \
LIMIT 10').show()


+--------------------+------------------+------------------+------------+
|        channeltitle|          avglikes|       avgdislikes|no_of_videos|
+--------------------+------------------+------------------+------------+
| Sky Sports Football|11241.897959183674|288.04761904761904|          84|
|    The United Stand| 10594.61809045226| 472.4321608040201|          67|
|                 WWE| 42906.43023255814| 1518.796511627907|          49|
|            BT Sport|         15977.712|           454.288|          46|
|Jay's Virtual Pub...|189.58695652173913| 19.42391304347826|          21|
|Mark Goldbridge T...| 3839.183908045977|125.05747126436782|          19|
|           FORMULA 1| 53358.07246376811|1160.1739130434783|          17|
|                 NFL|23722.316666666666|1011.2333333333333|          15|
|             Thogden|            8972.4|             454.3|          13|
|UFC - Ultimate Fi...|20809.217391304348| 734.0652173913044|          13|
+--------------------+----------------

In [ ]:
spark.sql('SELECT channeltitle, AVG(likes) as avglikes, AVG(dislikes) as avgdislikes, AVG(view_count) as avgviews,COUNT(DISTINCT video_id) as no_of_videos FROM youtube_gb \
GROUP BY channeltitle \
HAVING avgviews > 100000 AND no_of_videos > 15 \
ORDER BY avglikes DESC \
LIMIT 10').show()


+--------------------+------------------+------------------+------------------+------------+
|        channeltitle|          avglikes|       avgdislikes|          avgviews|no_of_videos|
+--------------------+------------------+------------------+------------------+------------+
|           FORMULA 1| 53358.07246376811|1160.1739130434783|2325830.3333333335|          17|
|                 WWE| 42906.43023255814| 1518.796511627907|1622012.3023255814|          49|
|            BT Sport|         15977.712|           454.288|         965365.96|          46|
| Sky Sports Football|11241.897959183674|288.04761904761904| 625022.4149659864|          84|
|    The United Stand| 10594.61809045226| 472.4321608040201|362126.47236180905|          67|
|Mark Goldbridge T...| 3839.183908045977|125.05747126436782|138192.06896551725|          19|
+--------------------+------------------+------------------+------------------+------------+



In [ ]:
spark.sql('SELECT channeltitle, AVG(likes) as avglikes, AVG(dislikes) as avgdislikes, AVG(view_count) as avgviews,COUNT(DISTINCT video_id) as no_of_videos \
FROM youtube_gb \
GROUP BY channeltitle \
HAVING avgviews > 100000 \
AND no_of_videos > 15 \
ORDER BY avgdislikes DESC \
LIMIT 10').show()

+--------------------+------------------+------------------+------------------+------------+
|        channeltitle|          avglikes|       avgdislikes|          avgviews|no_of_videos|
+--------------------+------------------+------------------+------------------+------------+
|                 WWE| 42906.43023255814| 1518.796511627907|1622012.3023255814|          49|
|           FORMULA 1| 53358.07246376811|1160.1739130434783|2325830.3333333335|          17|
|    The United Stand| 10594.61809045226| 472.4321608040201|362126.47236180905|          67|
|            BT Sport|         15977.712|           454.288|         965365.96|          46|
| Sky Sports Football|11241.897959183674|288.04761904761904| 625022.4149659864|          84|
|Mark Goldbridge T...| 3839.183908045977|125.05747126436782|138192.06896551725|          19|
+--------------------+------------------+------------------+------------------+------------+



In [ ]:
# most comments - looking at channels
spark.sql('SELECT channeltitle, AVG(likes) as avglikes, AVG(dislikes) as avgdislikes, AVG(view_count) as avgviews,AVG(comment_count) as avgcomment,COUNT(DISTINCT video_id) as no_of_videos \
FROM youtube_gb \
GROUP BY channeltitle \
HAVING avgviews > 100000 \
AND no_of_videos > 15 \
ORDER BY avgcomment DESC \
LIMIT 10').show()

+--------------------+------------------+------------------+------------------+------------------+------------+
|        channeltitle|          avglikes|       avgdislikes|          avgviews|        avgcomment|no_of_videos|
+--------------------+------------------+------------------+------------------+------------------+------------+
|           FORMULA 1| 53358.07246376811|1160.1739130434783|2325830.3333333335| 5211.449275362319|          17|
|                 WWE| 42906.43023255814| 1518.796511627907|1622012.3023255814|3283.8720930232557|          49|
|            BT Sport|         15977.712|           454.288|         965365.96|          1918.976|          46|
| Sky Sports Football|11241.897959183674|288.04761904761904| 625022.4149659864|1892.7142857142858|          84|
|    The United Stand| 10594.61809045226| 472.4321608040201|362126.47236180905| 992.2311557788945|          67|
|Mark Goldbridge T...| 3839.183908045977|125.05747126436782|138192.06896551725|280.82758620689657|      

In [ ]:
# least comments but still big channels
spark.sql('SELECT channeltitle, AVG(likes) as avglikes, AVG(dislikes) as avgdislikes, AVG(view_count) as avgviews,AVG(comment_count) as avgcomment,COUNT(DISTINCT video_id) as no_of_videos \
FROM youtube_gb \
GROUP BY channeltitle \
HAVING avgviews > 1000000 \
AND no_of_videos > 20 \
ORDER BY avgcomment asc \
LIMIT 10').show()

+------------+-----------------+-----------------+------------------+------------------+------------+
|channeltitle|         avglikes|      avgdislikes|          avgviews|        avgcomment|no_of_videos|
+------------+-----------------+-----------------+------------------+------------------+------------+
|         WWE|42906.43023255814|1518.796511627907|1622012.3023255814|3283.8720930232557|          49|
+------------+-----------------+-----------------+------------------+------------------+------------+



In [ ]:
# most comments for a video (do they correlate with most viewed/liked videos) - they do
spark.sql('SELECT channeltitle, title, publishedat,comment_count, likes, dislikes, trending_date \
FROM youtube_gb \
ORDER BY comment_count desc \
LIMIT 10').show(truncate=False)

+--------------+--------------------------------------------------------------------------------------------------+--------------------+-------------+--------+--------+--------------------+
|channeltitle  |title                                                                                             |publishedat         |comment_count|likes   |dislikes|trending_date       |
+--------------+--------------------------------------------------------------------------------------------------+--------------------+-------------+--------+--------+--------------------+
|Big Hit Labels|BTS (방탄소년단) 'Dynamite' Official MV                                                           |2020-08-21T03:58:10Z|5748647      |14678091|576853  |2020-08-24T00:00:00Z|
|Big Hit Labels|BTS (방탄소년단) 'Dynamite' Official MV                                                           |2020-08-21T03:58:10Z|5744360      |15246514|646350  |2020-08-26T00:00:00Z|
|Big Hit Labels|BTS (방탄소년단) 'Dynamite' Official MV          